In [1]:
import pandas as pd
import os
!pip install datasets
import torch
from datasets import Dataset

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"torch.device: {device}")

torch.device: cuda


In [3]:
    from google.colab import drive 
    drive.mount('/content/gdrive')
    folder_path = "gdrive/MyDrive/data/"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
# get list of files in folder
file_list = os.listdir(folder_path)

# create empty list to hold dataframes
df_list = []

# loop through files in folder
for file in file_list:
    # check if file is a CSV
    if file.endswith('.csv'):
        # read CSV file into a pandas dataframe
        df = pd.read_csv(os.path.join(folder_path, file))
        # append dataframe to list
        df_list.append(df)

# concatenate all dataframes in list into a single dataframe
df = pd.concat(df_list, axis=0, ignore_index=True)

<ipython-input-4-e19cb10073e8>:12: DtypeWarning: Columns (17,58,60,62) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(folder_path, file))
<ipython-input-4-e19cb10073e8>:12: DtypeWarning: Columns (58) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(folder_path, file))
<ipython-input-4-e19cb10073e8>:12: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(folder_path, file))
<ipython-input-4-e19cb10073e8>:12: DtypeWarning: Columns (17,66,68,70) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(folder_path, file))
<ipython-input-4-e19cb10073e8>:12: DtypeWarning: Columns (56,58,60,62) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(folder_path, file))
<ipython-input-4-e19cb10073e8>:12

In [5]:
df["new_text"] = "<UTTERANCE_START>" + df["speaker"] + " says: '" + df["text"] + "' to " + df["speaker_replied_to"] + " <UTTERANCE_END>"
#j_columns = [col for col in df.columns if col.startswith('votes_side_j_')]
#df = df[['case_id', 'new_text'] + j_columns]
df = df[['case_id', 'new_text', 'win_side']]


In [6]:
grouped_df = df.groupby('case_id')['new_text'].apply(lambda x: ','.join(x)).reset_index()
#justices = df[["case_id"] + j_columns].drop_duplicates(keep='first')
#df1 = pd.merge(grouped_df, justices, left_on='case_id', right_on='case_id', how='left').dropna(axis='columns', how='any')

df1 = pd.merge(grouped_df, df[['case_id','win_side']].drop_duplicates(keep='first'), left_on='case_id', right_on='case_id', how='left').dropna(axis='rows', how='any')


In [7]:
from datasets import *
dataset = Dataset.from_pandas(df1.drop('case_id', axis=1), preserve_index = False)

dataset = dataset.train_test_split(test_size=0.3, shuffle=True)

dataset_test_valid = dataset['test'].train_test_split(test_size=0.5, shuffle=True)

dataset = DatasetDict({
    'train': dataset['train'],
    'test': dataset_test_valid['test'],
    'validation': dataset_test_valid['train']})

In [8]:
df1

,case_id,new_text,win_side
0,1955_10,<UTTERANCE_START>herbert_s_thatcher says: 'You...,0.0
1,1955_102,<UTTERANCE_START>j__hugo_l_black says: 'May I ...,0.0
2,1955_110,<UTTERANCE_START>j__hugo_l_black says: 'What j...,1.0
3,1955_111,<UTTERANCE_START><INAUDIBLE> says: '(Inaudible...,1.0
4,1955_112,<UTTERANCE_START>daniel_duke says: 'This is Da...,1.0
...,...,...,...
1793,2019_19-631,<UTTERANCE_START>malcolm_l_stewart says: 'Than...,0.0
1794,2019_19-635,<UTTERANCE_START>jay_alan_sekulow says: 'Thank...,0.0
1795,2019_19-67,<UTTERANCE_START>eric_j_feigin says: 'Thank yo...,1.0
1796,2019_19-7,<UTTERANCE_START>j__ruth_bader_ginsburg says: ...,1.0


In [9]:
labels = [label for label in dataset['train'].features.keys() if label not in ['case_id', 'new_text']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

!pip install transformers
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(examples):
  # take a batch of texts
  text = examples["new_text"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()
  
  return encoding

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)

Map:   0%|          | 0/1255 [00:00<?, ? examples/s]

Map:   0%|          | 0/270 [00:00<?, ? examples/s]

Map:   0%|          | 0/269 [00:00<?, ? examples/s]

In [11]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [12]:
!pip install transformers==4.28.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
batch_size = 8
metric_name = "f1"

from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

In [14]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [15]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,No log,0.701669,0.438662,0.468757,0.438662
2,No log,0.691593,0.550186,0.500000,0.550186
3,No log,0.703128,0.550186,0.500000,0.550186
4,0.670700,0.749750,0.553903,0.509409,0.553903
5,0.670700,0.742467,0.542751,0.513597,0.542751


{'eval_loss': 0.7497504949569702,
 'eval_f1': 0.5539033457249071,
 'eval_roc_auc': 0.5094092025910208,
 'eval_accuracy': 0.5539033457249071,
 'eval_runtime': 2.0772,
 'eval_samples_per_second': 129.504,
 'eval_steps_per_second': 16.369,
 'epoch': 5.0}